# VariantFormer: Variant-to-Expression Prediction

## Predicting the Functional Impact of Genetic Variants on Gene Expression

### Executive Summary

VariantFormer is a state-of-the-art deep learning framework that predicts how genetic variants affect gene expression across diverse tissues and populations. This notebook demonstrates the core functionality of the VariantFormer system, showcasing how single nucleotide variants (SNVs) can be analyzed to understand their potential impact on cellular function.

### Innovation

VariantFormer addresses several critical limitations of existing approaches:

1. **Multi-tissue Analysis**: Predicts variant effects across 62 tissues and cell types
2. **Population-aware**: Considers genetic background from major world populations
3. **Mechanistic Understanding**: Models both cis-regulatory elements (CREs) and gene sequences
---


## Methodology Overview

VariantFormer employs a sophisticated **transformer-based architecture** that integrates DNA sequences around genes and regulatory elements.

### Scoring Strategies

#### 1. Expression Prediction Scores
- **Direct Output**: Predicted gene expression values
- **Comparison**: Reference (wild-type) vs. heterozygous vs. homozygous variant genotypes
- **Tissue-specific**: Separate predictions for each tissue type

#### 2. Log2 Fold Change (log2fc) Regulation Scores as eQTLs
- **Population Comparison**: Analyzes expression differences across populations
- **Reference Comparison**: Compares variant expression to reference genome expression  
- **Directional Interpretation**: 
  - **Positive log2fc (>0)**: Gene upregulation - variant increases expression
  - **Negative log2fc (<0)**: Gene downregulation - variant decreases expression  
  - **log2fc ≈ 0**: Minimal effect - variant has little impact
- **Population Weighting**: Aggregates scores across populations using allele frequency weights
- **Magnitude**: Larger absolute values indicate stronger regulatory effects

## Initialize VariantFormer

The `v4_ag` model supports all genes (protein-coding and non-coding). Use `v4_pcg` for protein-coding genes only.


In [1]:
# Essential imports
import sys
from pathlib import Path
import warnings

warnings.filterwarnings("ignore")

sys.path.append(str(Path.cwd().parent))

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from processors.variantprocessor import VariantProcessor


In [2]:
# Initialize VariantFormer
print("🚀 Initializing VariantFormer Variant Processor...")
model_class = 'v4_ag' # model class can be 'v4_ag', 'v4_pcg'. AG model is all-genes model trained on both protein-coding and non-coding genes.
# model_class = 'v4_pcg' # Uncomment to use the PCG model
vep = VariantProcessor(model_class=model_class)

print("✅ System initialized!")
print(f"🌍 Populations: {', '.join(vep.populations)}")
print(f"🧬 Tissues: {len(vep.tissue_vocab)} available")

🚀 Initializing VariantFormer Variant Processor...
✅ System initialized!
🌍 Populations: REF_HG38, EAS, EUR, AFR, SAS, AMR
🧬 Tissues: 62 available


## Define Variant for Analysis

Specify variant coordinates, alleles, target tissue, and gene ID.


In [3]:
# Define variants for analysis
print("🧬 Creating multi-variant dataset...")

multi_variants = {
    "chr": ["chr13"],
    "pos": [
        113978728,
    ],
    "ref": [
        "A",
    ],
    "alt": ["G"],
    "tissue": ["whole blood"],
    "gene_id": ["ENSG00000185989.10"],
}

variant_df = pd.DataFrame(multi_variants)
print(variant_df.to_string(index=False))


🧬 Creating multi-variant dataset...
  chr       pos ref alt      tissue            gene_id
chr13 113978728   A   G whole blood ENSG00000185989.10


## Run Predictions

Generate expression predictions across populations and zygosity states.


In [4]:
# Run variant predictions
print("🔬 Running VariantFormer variant analysis...")
output_dir = "/tmp/vep_multi_output"
# Predict expression effects for all variants
raw_predictions = vep.predict(var_df=variant_df, output_dir=output_dir)

2025-11-02 21:27:46 - processors.variantprocessor - INFO - Initializing Variant Processor...
2025-11-02 21:27:46 - processors.multi_datasets_loader - INFO - Loading gene annotations...
2025-11-02 21:27:46 - processors.multi_datasets_loader - INFO - Loading CRE annotations...


🔬 Running VariantFormer variant analysis...


2025-11-02 21:27:47 - processors.variantprocessor - INFO - Loading variants...
2025-11-02 21:27:47 - processors.multi_datasets_loader - INFO - Loaded 1 variants
2025-11-02 21:27:47 - processors.variantprocessor - INFO - Loaded 1 variants for processing
2025-11-02 21:27:47 - processors.variantprocessor - INFO - Mapped 1 gene-variant pairs
2025-11-02 21:27:47 - processors.variantprocessor - INFO - Loading BPE encoder...
2025-11-02 21:27:47 - processors.variantprocessor - INFO - Loading model...
2025-11-02 21:27:47 - processors.model_manager - INFO - Loading Seq2Reg model...


Loaded BPE vocabulary from /work/vocabs/bpe_vocabulary_500_using_huggingface.json


2025-11-02 21:27:48 - processors.model_manager - INFO - Loading Seq2Reg gene model...
2025-11-02 21:27:48 - processors.model_manager - INFO - Creating Seq2Gene model...
2025-11-02 21:27:53 - processors.model_manager - INFO - Model class: <class 'seq2gene.model_combined_modulator.Seq2GenePredictorCombinedModulator'>
2025-11-02 21:27:53 - processors.model_manager - INFO - Model architecture:
2025-11-02 21:27:53 - processors.model_manager - INFO - Model: Seq2GenePredictorCombinedModulator
2025-11-02 21:27:53 - processors.model_manager - INFO -   start_tkn: 96,768 params
2025-11-02 21:27:53 - processors.model_manager - INFO -   cre_tokenizer: 31,826,153 params
2025-11-02 21:27:53 - processors.model_manager - INFO -   gene_tokenizer: 31,826,153 params
2025-11-02 21:27:53 - processors.model_manager - INFO -   gene_map: 787,968 params
2025-11-02 21:27:53 - processors.model_manager - INFO -   cre_map: 787,968 params
2025-11-02 21:27:53 - processors.model_manager - INFO -   combined_modulator: 

Predicting: |          | 0/? [00:00<?, ?it/s]

2025-11-02 21:27:58 - utils.assets - INFO - Downloading from S3: s3://czi-variantformer/model/common/reference_genomes/cres_seqs_manifest.parquet
2025-11-02 21:27:58 - utils.assets - INFO - Downloading from S3: s3://czi-variantformer/model/common/reference_genomes/cres_seqs_manifest.parquet
2025-11-02 21:27:58 - utils.assets - INFO - Loading parquet file: /tmp/tmps7kh_4yf/model/common/reference_genomes/cres_seqs_manifest.parquet
2025-11-02 21:27:58 - utils.assets - INFO - Validated schema - found columns: {'chromosome', '__index_level_0__', 'file_path', 'population'}
2025-11-02 21:27:58 - utils.assets - INFO - Downloading from S3: s3://czi-variantformer/model/common/reference_genomes/data_split/eur/cres/data_split/HG00096_chr13.pkl.gz
2025-11-02 21:27:58 - utils.assets - INFO - Loading parquet file: /tmp/tmps7kh_4yf/model/common/reference_genomes/cres_seqs_manifest.parquet
2025-11-02 21:27:58 - utils.assets - INFO - Loading parquet file: /tmp/tmps7kh_4yf/model/common/reference_genomes/

In [5]:
raw_predictions.head()

,chrom,pos,ref,alt,genes,tissues,variant_type,population,sample_name,zygosity,gene_exp,gene_emb,gene_token_embedding,cre_token_embedding
0,chr13,113978728,A,G,ENSG00000185989.10,whole blood,Gene overlap only,REF_HG38,hg38,2,3.182971,"[4.90625, 0.16699219, -0.5546875, -2.546875, 6...","[0.87109375, -0.8671875, -1.0078125, -0.578125...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,chr13,113978728,A,G,ENSG00000185989.10,whole blood,Gene overlap only,REF_HG38,hg38,1,3.182971,"[4.875, 0.19042969, -0.5234375, -2.5625, 6.281...","[0.80859375, -0.67578125, -0.45703125, 0.10937...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,chr13,113978728,A,G,ENSG00000185989.10,whole blood,Gene overlap only,REF_HG38,hg38,0,3.182971,"[4.90625, 0.18164062, -0.5390625, -2.5625, 6.2...","[0.96484375, -0.875, -0.7265625, -0.3671875, 2...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,chr13,113978728,A,G,ENSG00000185989.10,whole blood,Gene overlap only,EAS,HG00404,2,3.454005,"[4.625, 0.19140625, 2.515625, 0.12695312, 5.37...","[2.9375, -1.4765625, -0.328125, -1.6875, 2.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,chr13,113978728,A,G,ENSG00000185989.10,whole blood,Gene overlap only,EAS,HG00404,1,3.454005,"[4.625, 0.19140625, 2.515625, 0.12695312, 5.37...","[2.9375, -1.4765625, -0.328125, -1.6875, 2.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


**Prediction Output Schema**
- `chrom`, `pos`, `ref`, `alt`: Variant coordinates and alleles
- `genes`: Ensembl Gene ID
- `tissue`: Tissue name
- `population`: Population code (REF_HG38, EAS, EUR, AFR, SAS, AMR)
- `sample_name`: 1000 Genomes sample identifier
- `zygosity`: 0 (homozygous ref), 1 (heterozygous), 2 (homozygous alt)
- `gene_exp`: Predicted gene expression level for the gene in the specified tissue and sample
- `variant_type`: Gene Overlap / CRE Overlap
- `gene_emb`: Embedding of the gene conditioned on tissue and regulatory context from the last transformer layer
- `gene_token_embedding`:  Embedding of the token that overlaps with the variant position from the last transformer layer
- `cre_token_embedding`: Embeddings of regulatory elements overlapping with the variant position from the last transformer layer


## Format Scores and Calculate eQTL Effects

Transform raw predictions into structured format and compute log2 fold changes across populations.


In [6]:
# Format and score the predictions
print("📊 Formatting and scoring predictions...")

# Step 1: Format scores
formatted_scores = vep.format_scores(raw_predictions)


# Step 2: Calculate eQTL scores (log2fc-based)
final_results = vep.eqtl_scores(formatted_scores)
print(f"✅ Calculated regulation scores for {len(final_results)} variants")
final_results.head()

📊 Formatting and scoring predictions...
   REF_HG38-0-exp  AFR-2-exp  AMR-2-exp  EAS-2-exp  EUR-2-exp  REF_HG38-2-exp  \
0        3.182971   3.303087   3.333206   3.454005   3.242956        3.182971   

   SAS-2-exp           variant_id               genes      tissues  ...  \
0   3.273004  chr13_113978728_A_G  ENSG00000185989.10  whole blood  ...   

  VF-EAS-2-exp-log2fc VF-EUR-2-exp-log2fc VF-REF_HG38-2-exp-log2fc  \
0            0.117896            0.026935                      0.0   

   VF-SAS-2-exp-log2fc        AF    AF_EUR    AF_EAS    AF_AMR    AF_SAS  \
0             0.040241  0.282324  0.368088  0.354701  0.354082  0.327787   

     AF_AFR  
0  0.104143  

[1 rows x 26 columns]
✅ Calculated regulation scores for 1 variants


,variant_id,genes,tissues,ref,alt,chr,pos,VF-agg-log2fc-weighted,VF-AFR-2-exp-log2fc,VF-AMR-2-exp-log2fc,VF-EAS-2-exp-log2fc,VF-EUR-2-exp-log2fc,VF-REF_HG38-2-exp-log2fc,VF-SAS-2-exp-log2fc
0,chr13_113978728_A_G,ENSG00000185989.10,whole blood,A,G,chr13,113978728,0.062333,0.053441,0.066536,0.117896,0.026935,0.0,0.040241


**Population-specific log2 fold changes** (homozygous alt vs. ref):
- `VF-{POP}-2-exp-log2fc`: Log2 fold change in expression for the homozygous alternative allele in the background genome compared to reference' (Background POP: AFR, EUR, EAS, SAS, AMR)
- `VF-REF_HG38-2-exp-log2fc`: Expression change in reference genome
- `VF-agg-log2fc-weighted`: Log2 fold change in expression for the homozygous alternative allele in the HG38 reference genome compared to reference homozygous ref


## Score Interpretation

- **log2fc > 0**: Upregulation (≥1.0: strong, ≥0.5: moderate)
- **log2fc ≈ 0**: Neutral effect
- **log2fc < 0**: Downregulation (≤-0.5: moderate, ≤-1.0: strong)


## Key Findings & Applications

### Variant-level Insights
VariantFormer enables simultaneous analysis of variant sets to:
- **Prioritize variants** by regulatory impact
- **Compare effects** across different genomic loci  
- **Identify tissue-specific** regulatory variants
- **Population-stratified** variant interpretation

### Clinical Applications
- Focus on functionally relevant variants  
- Population-aware variant interpretation  
- Identify expression-modulating variants  
- Prioritize causal variants from association studies  



---

# VCF-Based Analysis

VariantFormer can analyze variants insilico in individual genotype context using VCF files for personalized genomic insights.


In [7]:
# Example: VCF-based Analysis
print("🧬 VCF Integration Example")
print("=" * 40)

# VCF files contain individual genotype information
# This enables more precise analysis based on actual sample data

🧬 VCF Integration Example


In [8]:
# For VCF analysis, we can reuse the existing VariantProcessor
print("🔄 Using existing VariantFormer system for VCF analysis...")
print("📁 VCF files enable individual-specific genomic analysis")

🔄 Using existing VariantFormer system for VCF analysis...
📁 VCF files enable individual-specific genomic analysis


## Define VCF Parameters

Specify VCF file path, sample name, and variant coordinates for individual-specific analysis.


In [9]:
model_class = 'v4_ag' # model class can be 'v4_ag', 'v4_pcg'. AG model is all-genes model trained on both protein-coding and non-coding genes.
# model_class = 'v4_pcg' # Uncomment to use the PCG model
vep = VariantProcessor(model_class=model_class)

In [10]:
# Example VCF integration parameters
vcf_path = os.path.join(str(Path.cwd().parent),"_artifacts/HG00096.vcf.gz") # Change path as needed
sample_name = "HG00096"  # European sample from 1000 Genomes Project


# Same variant for insilico analysis. Alt allele will be placed in the sample genotype context at that specific position. 
# Ref is from reference genome.
vcf_variant_data = {
    "chr": ["chr13"],
    "pos": [113978728],
    "ref": ["A"],
    "alt": ["G"],
    "tissue": ["whole blood"],
    "gene_id": ["ENSG00000185989.10"],
}

vcf_variant_df = pd.DataFrame(vcf_variant_data)
print("\n🧬 Analyzing the same variant with individual genotype data:")
print(vcf_variant_df.to_string(index=False))


🧬 Analyzing the same variant with individual genotype data:
  chr       pos ref alt      tissue            gene_id
chr13 113978728   A   G whole blood ENSG00000185989.10


## Run VCF-Based Predictions

Generate predictions with individual genotype context from the VCF file.


In [11]:
# Run VariantFormer with VCF integration
print("\n🔬 Running VariantFormer analysis with VCF integration...")
print("⏳ Processing individual genotype data...")

vcf_predictions = vep.predict(
    var_df=vcf_variant_df,
    output_dir="/tmp/vep_output_vcf",
    vcf_path=vcf_path,
    sample_name=sample_name,
)

print("✅ VCF-based predictions completed!")


2025-11-02 21:28:36 - processors.variantprocessor - INFO - Initializing Variant Processor...
2025-11-02 21:28:36 - processors.multi_datasets_loader - INFO - Loading gene annotations...
2025-11-02 21:28:36 - processors.multi_datasets_loader - INFO - Loading CRE annotations...



🔬 Running VariantFormer analysis with VCF integration...
⏳ Processing individual genotype data...


2025-11-02 21:28:37 - processors.variantprocessor - INFO - Loading variants...
2025-11-02 21:28:37 - processors.multi_datasets_loader - INFO - Loaded 1 variants
2025-11-02 21:28:37 - processors.variantprocessor - INFO - Loaded 1 variants for processing
2025-11-02 21:28:37 - processors.variantprocessor - INFO - Mapped 1 gene-variant pairs
2025-11-02 21:28:37 - processors.variantprocessor - INFO - Loading BPE encoder...
2025-11-02 21:28:37 - processors.variantprocessor - INFO - Loading model...
2025-11-02 21:28:37 - processors.model_manager - INFO - Loading Seq2Reg model...


Loaded BPE vocabulary from /work/vocabs/bpe_vocabulary_500_using_huggingface.json


2025-11-02 21:28:37 - processors.model_manager - INFO - Loading Seq2Reg gene model...
2025-11-02 21:28:37 - processors.model_manager - INFO - Creating Seq2Gene model...
2025-11-02 21:28:42 - processors.model_manager - INFO - Model class: <class 'seq2gene.model_combined_modulator.Seq2GenePredictorCombinedModulator'>
2025-11-02 21:28:42 - processors.model_manager - INFO - Model architecture:
2025-11-02 21:28:42 - processors.model_manager - INFO - Model: Seq2GenePredictorCombinedModulator
2025-11-02 21:28:42 - processors.model_manager - INFO -   start_tkn: 96,768 params
2025-11-02 21:28:42 - processors.model_manager - INFO -   cre_tokenizer: 31,826,153 params
2025-11-02 21:28:42 - processors.model_manager - INFO -   gene_tokenizer: 31,826,153 params
2025-11-02 21:28:42 - processors.model_manager - INFO -   gene_map: 787,968 params
2025-11-02 21:28:42 - processors.model_manager - INFO -   cre_map: 787,968 params
2025-11-02 21:28:42 - processors.model_manager - INFO -   combined_modulator: 

Predicting: |          | 0/? [00:00<?, ?it/s]

2025-11-02 21:28:47 - utils.assets - INFO - Downloading from S3: s3://czi-variantformer/model/common/cres_all_genes_manifest.parquet
2025-11-02 21:28:48 - utils.assets - INFO - Loading parquet file: /tmp/tmpg74yoati/model/common/reference_genomes/cres_seqs_manifest.parquet
2025-11-02 21:28:48 - utils.assets - INFO - Validated schema - found columns: {'chromosome', '__index_level_0__', 'file_path', 'population'}
2025-11-02 21:28:48 - utils.assets - INFO - Downloading from S3: s3://czi-variantformer/model/common/reference_genomes/data_split/hg38/cres/data_split/hg38_chr13.pkl.gz
2025-11-02 21:28:49 - utils.assets - INFO - Loading parquet file: /tmp/tmpvdu4y71l/model/common/cres_all_genes_manifest.parquet
2025-11-02 21:28:49 - utils.assets - INFO - Validated schema - found columns: {'file_path', 'gene_id'}
2025-11-02 21:28:49 - utils.assets - INFO - Downloading from S3: s3://czi-variantformer/model/common/cres_all_genes/ENSG00000185989.10/gene_vocab.csv
2025-11-02 21:28:49 - utils.assets 

✅ VCF-based predictions completed!


## VCF Output Format

**Expression columns:**
- `REF_HG38-{0,1,2}-exp`: Expression in reference genome (homozygous ref, het, homozygous alt)
- `SAMPLE-0-exp`: Predicted expression for the individual without perturbation at that locus')
- `SAMPLE-1-exp`: Predicted expression for the individual with heterozygous alternate at that locus')
- `SAMPLE-2-exp`: Predicted expression for the individual with homozygous alternate at that locus')

In [12]:
# Format scores for VCF-based predictions
vcf_formatted_scores = vep.format_scores(vcf_predictions)
vcf_formatted_scores.head()

gt-exp,variant_id,genes,tissues,chr,pos,ref,alt,variant_type,REF_HG38-0-exp,REF_HG38-1-exp,REF_HG38-2-exp,SAMPLE-0-exp,SAMPLE-1-exp,SAMPLE-2-exp
0,chr13_113978728_A_G,ENSG00000185989.10,whole blood,chr13,113978728,A,G,Gene overlap only,3.182971,3.182971,3.182971,3.242956,3.242956,3.242956


In [13]:
# Apply eQTL scoring to VCF results
vcf_final_results = vep.eqtl_scores(vcf_formatted_scores)

print("✅ VCF-based eQTL scoring completed!")

vcf_final_results.head()

✅ VCF-based eQTL scoring completed!


gt-exp,variant_id,genes,tissues,ref,alt,chr,pos,VF-REF_HG38-2-exp-log2fc,VF-SAMPLE-2-exp-log2fc
0,chr13_113978728_A_G,ENSG00000185989.10,whole blood,A,G,chr13,113978728,0.0,0.026935


**column interpretations**

- `VF-REF_HG38-2-exp-log2fc`: Log2 fold change in expression for the homozygous alternative allele in the HG38 reference genome compared to reference homozygous ref
- `VF-SAMPLE-2-exp-log2fc`: Log2 fold change in expression for the homozygous alternative allele in the individual sample compared to reference
